In [1]:
import numpy as np
import re
import itertools
from keras.utils import np_utils
from keras.models import Sequential
from keras.layers import Dense, SimpleRNN, Activation
from keras.callbacks import ModelCheckpoint
from random import randint
import matplotlib.pyplot as plt
import keras

Using Theano backend.
Using gpu device 0: GeForce GT 640M LE (CNMeM is disabled, cuDNN 5005)


In [9]:
charToInt = {}
intToChar = {}
def getEncoding(merged_data):
    indx = 0
    fin = []
    for x in merged_data:
        if x not in charToInt:
            charToInt[x] = indx
            intToChar[indx] = x
            indx += 1
        fin.append(charToInt[x])
    return fin
            
def loadMusic(file):
    f = open(file, 'r')
    text = f.read()
    text = text.replace("\r", "")
    words = re.split(r"(\s+)", text)
    new_words = words #[x for x in words if (x != '<start>' and x != '<end>')] # get rid of <start> and <end>
    data = []
    [data.append(list(w)) for w in new_words]
    merged_data = np.asarray(list(itertools.chain.from_iterable(data)))  
#    s = set(merged_data)
#    char_int = [ch:i for i,ch in enumerate(s)] # encode characters to integers
#    int_char = [i:ch for i,ch in enumerate(s)] # encode integers to characters
    encoded_data = getEncoding(merged_data)#[ord(x) for x in merged_data] # assing ascii labels to characters
    return encoded_data

def prepareData(data, t):
	dataX, dataY = [], []
	for i in range(len(data) - t - 1):
		a = data[i:(i+t)]
		dataX.append(a)
		dataY.append(data[i + t])
	return np.array(dataX), np.array(dataY)

def splitData(data, v):
    train_size = int(len(data[0]) * v) # v is between 0 and 1
    train, test = (data[0][0:train_size],data[1][0:train_size]), (data[0][train_size:len(data[0])],data[1][train_size:len(data[0])])
    return train, test

def sliceData(data, s):
    dataX, dataY = data[0], data[1]
    X, Y = [], []
    windowStride = 5
    for i in range((len(data[0])-s - 1) / windowStride):
        seq = i * windowStride #randint(1,len(data[0])-20)
        sliceX = dataX[seq:seq+s]
        sliceY = dataY[seq+s-1]
        X.append(sliceX)
        Y.append(sliceY)
    data = (X, Y)
    return splitData(data, 0.8)

In [10]:
file = 'input.txt'
abc_list = loadMusic(file) # read in data
lvl = np.min(abc_list)
abc_list = abc_list - lvl
vocab = np.max(abc_list)
print lvl


data = prepareData(abc_list, 1) # prepare inputs and targets
#dataslice = sliceData(data, s = 2) # slice into sequences of length s

train_seq, test_seq = sliceData(data, 30)
y_train = train_seq[1]
y_test = test_seq[1]

print len(y_train), len(train_seq[0])

# one-hot encoding targets
X_train = np_utils.to_categorical(train_seq[0], vocab+1)
X_test = np_utils.to_categorical(test_seq[0], vocab+1)
Y_train = np_utils.to_categorical(train_seq[1], vocab+1)
Y_test = np_utils.to_categorical(test_seq[1], vocab+1)
print X_train.shape, Y_train.shape, X_test.shape, Y_test.shape

seq = 30; # sequence length
X_train = X_train[0:int(X_train.shape[0]/seq)*seq][:]
Y_train = Y_train[0:int(Y_train.shape[0])][:]
X_test = X_test[0:int(X_test.shape[0]/seq)*seq][:]
Y_test = Y_test[0:int(Y_test.shape[0])][:]
print X_train.shape, Y_train.shape, X_test.shape, Y_test.shape

X_train = X_train.reshape(int(X_train.shape[0]/seq), seq, X_train.shape[1]) # reshape input for RNN
X_test = X_test.reshape(int(X_test.shape[0]/seq), seq, X_test.shape[1])
Y_train = Y_train.reshape(int(Y_train.shape[0]), Y_train.shape[1]) # reshape input for RNN
Y_test = Y_test.reshape(int(Y_test.shape[0]), Y_test.shape[1])


0
80229 80229
(2406870, 93) (80229, 93) (601740, 93) (20058, 93)
(2406870, 93) (80229, 93) (601740, 93) (20058, 93)


In [11]:
print X_train.shape, Y_train.shape, X_test.shape, Y_test.shape

(80229, 30, 93) (80229, 93) (20058, 30, 93) (20058, 93)


In [12]:
# for x in X_train[0]:
#     print (intToChar[np.argmax(x)])
# print np.argmax(Y_train[0])
# print (intToChar[np.argmax(Y_train[0])])

In [13]:
hid_neurons = 150
nb_classes = np.shape(Y_train)[-1]


#%% create model
model = Sequential()
model.add(SimpleRNN(output_dim = hid_neurons,
                    init='glorot_uniform', inner_init='orthogonal', activation='relu',
                    W_regularizer=None, U_regularizer=None, b_regularizer=None,
                    dropout_W=0.0, dropout_U=0.0, return_sequences = False,
                    input_shape = X_train.shape[1:]))
model.add(Dense(output_dim = nb_classes))
model.add(Activation('softmax'))
model.summary()

adam_op = keras.optimizers.Adam(lr=0.001, beta_1=0.9, beta_2=0.999, epsilon=1e-08, decay=0.00001)


model.compile(loss='categorical_crossentropy',
              optimizer=adam_op,
              metrics=['accuracy'])
#%% train model and save weights
filepath="weights-improvement-{epoch:02d}-{loss:.4f}.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor='loss', verbose=1, save_best_only=True, mode='min')
callbacks_list = [checkpoint]


____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
simplernn_2 (SimpleRNN)          (None, 150)           36600       simplernn_input_2[0][0]          
____________________________________________________________________________________________________
dense_2 (Dense)                  (None, 93)            14043       simplernn_2[0][0]                
____________________________________________________________________________________________________
activation_2 (Activation)        (None, 93)            0           dense_2[0][0]                    
Total params: 50,643
Trainable params: 50,643
Non-trainable params: 0
____________________________________________________________________________________________________


In [37]:
model.fit(X_train, Y_train, nb_epoch = 10, shuffle = True)

Epoch 1/10
80229/80229 [==============================] - 43s - loss: 1.2506 - acc: 0.6122    
Epoch 2/10
80229/80229 [==============================] - 44s - loss: 1.2436 - acc: 0.6140    
Epoch 3/10
80229/80229 [==============================] - 43s - loss: 1.2358 - acc: 0.6163    
Epoch 4/10
80229/80229 [==============================] - 38s - loss: 1.2294 - acc: 0.6193    
Epoch 5/10
80229/80229 [==============================] - 38s - loss: 1.2187 - acc: 0.6219    
Epoch 6/10
80229/80229 [==============================] - 38s - loss: 1.2110 - acc: 0.6227    
Epoch 7/10
80229/80229 [==============================] - 47s - loss: 1.2058 - acc: 0.6244    
Epoch 8/10
80229/80229 [==============================] - 38s - loss: 1.2000 - acc: 0.6257    
Epoch 9/10
80229/80229 [==============================] - 38s - loss: 1.1931 - acc: 0.6284    
Epoch 10/10
80229/80229 [==============================] - 37s - loss: 1.1855 - acc: 0.6300    


In [38]:
inp = X_train[0, :, :]
inp.shape
import sys

def printInp(inp):
    print "================================"
    for x in inp:
        print intToChar[np.argmax(x)],
    print "\n++================================"

for i in range(1300):
    out = model.predict(np.reshape(inp, (1, 30, 93)))
    final = np.random.choice(np.arange(0, 93), p = out[0])
    sys.stdout.write(intToChar[final])
    nextInp = np_utils.to_categorical(final, nb_classes=93)
    inp = np.ma.row_stack((inp, nextInp))
    inp = inp[1:]
#     printInp(inp)

:id:hn-hornpipe-19
M:C|
K:Deor
AE ED|1 BA | B>c B>A | A/B/A/B/ cB cc|
d3 g2e|gec B2B|dcd ead|ede cAG|AF ED||
GG FA|d2 AB|ef ed|:|
<end>
<start>
X:10
T:Jamisre Gous cone va-phie
.D.C.
W:Lis Cooteur de la nan
Z:id:hn-mazurka-9
M:C|
K:D
Bdec =AADE | GFGE D2FA:|
B2GB GFED |: B2z z2z | E>E D>D | BB B>B | BA/B/ B/d/d/e/ G2B|c2B c3 :|
<end>
<start>
X:20
T:Lee Montin: Ia (one la Danellen
R:jig
H:See arcone Raet fu-me la de Gerbim?n Mors and Minny, #120157Ie |
c2c2 :|
<end>
<start>
X:89
T:Maring
T:h'u Maggie
R:Carolan
R:air
H:Tram.id hois bet Inam Auran.
H:Thartis bon 1
Z:Daven-c'a ca
W:ankon: Matc on-tr?mn Hon
O:France
A:Provence
C:?m
R:Maz2
O:
D/2
R:cerolay's
T:An pardion Gincont ReNtent wettete tEemine, The
T:Dant'ranae
H:Raelaul te jeandons on the Fithereite:transe
D:Morled le Gedyry Gurvi?
O:France
A:Chrnain
C:Trad.
<end>
Z:Pant
O:France
R:garigtice
H:The nou the Denais, The
T:Varcels
R:Plorigis
R:Mancendmoh
D:Faria triginaid tamin: Thee Huph Morlose, Tor
R:polka
Z:in:hu-marcheadgan'rok, T

In [ ]:
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
#plt.title('model loss for %d samples per category')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()

#plt.plot(history.history['acc'])
plt.plot(history.history['val_acc'])
#plt.title('model accuracy for %d samples per category' % samCat)
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.show()
#%% load weights


In [ ]:
filename = "weights-improvement-19-1.9435.hdf5"
model.load_weights(filename)
model.compile(loss='categorical_crossentropy', optimizer='rmsprop')

#%% generating music

seed = X_train[1]
pattern = seed
for i in range(0,1000):
    	x = np.reshape(pattern, (1, pattern.shape[0], pattern.shape[1]))
    	prediction = model.predict(x, verbose=1)
     	final = np.random.choice(np.arange(0, 256), p = prediction)
        print intToChar[final]
        
      

